In [1]:
# pip install tensorflow
# pip install keras
# pip install numpy
# pip install matplotlib
# pip install pandas
# pip install scikit-learn
# pip install opencv-contrib-python

ResNet50

In [3]:
# Libraries
import os
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Check Image Corruption

In [4]:
from PIL import UnidentifiedImageError

def check_images(directory):
    corrupted_images = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # This will raise an exception if the image is corrupted
            except (UnidentifiedImageError, IOError) as e:
                corrupted_images.append(file_path)
                print(f"Corrupted image: {file_path} - {e}")
    return corrupted_images

corrupted_images = check_images('images')
print(f"Total corrupted images: {len(corrupted_images)}")


Total corrupted images: 0


In [9]:
def preprocess_image(image_path, size=(224, 224)):
    try:
        img = Image.open(image_path)
        
        # Quality enhancement
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(2.0)  # Increase sharpness
        
        # Noise reduction
        image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        image = cv2.GaussianBlur(image, (5, 5), 0)
        img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Consistency
        img = img.resize(size, Image.LANCZOS)

        # Normalization
        img_array = np.array(img) / 255.0
    
        # Convert back to image
        img = Image.fromarray((img_array * 255).astype(np.uint8))
        
        return img
    except (UnidentifiedImageError, IOError) as e:
        print(f"Error processing image {image_path}: {e}")
        return None

input_dir = 'images_train_test_val/train'
output_dir = 'processed_images'

# Apply preprocessing and save to new directory
for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)
    if os.path.isdir(category_path):
        output_category_path = os.path.join(output_dir, category)
        if not os.path.exists(output_category_path):
            os.makedirs(output_category_path)
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            preprocessed_img = preprocess_image(image_path)
            if preprocessed_img:
                preprocessed_img.save(os.path.join(output_category_path, image_name))


In [5]:
train_dir = 'data/processed_images'
val_dir = 'data/images_train_test_val/validation'
test_dir = 'data/images_train_test_val/test'

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 7350 images belonging to 21 classes.
Found 2100 images belonging to 21 classes.
Found 1050 images belonging to 21 classes.


In [6]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
predictions = Dense(21, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

2024-07-19 12:23:45.495640: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-07-19 12:23:45.495672: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-07-19 12:23:45.495681: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-07-19 12:23:45.495875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-19 12:23:45.495887: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Define callbacks
checkpoint = ModelCheckpoint(
    'models/best_ResNet_model.keras', 
    monitor='val_loss', 
    save_best_only=True, 
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    mode='min',
    verbose=1,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stopping]
)


Epoch 1/50


2024-07-18 19:40:18.819977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/jay/anaconda3/envs/tf_m1_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - accuracy: 0.6862 - loss: 9.1536 
Epoch 1: val_loss improved from inf to 11.08850, saving model to models/best_ResNet_model.keras
115/115 ━━━━━━━━━━━━━━━━━━━━ 2152s 18s/step - accuracy: 0.6877 - loss: 9.1462 - val_accuracy: 0.0486 - val_loss: 11.0885
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.9850 - loss: 7.0426 
Epoch 2: val_loss improved from 11.08850 to 9.87264, saving model to models/best_ResNet_model.keras
115/115 ━━━━━━━━━━━━━━━━━━━━ 2032s 18s/step - accuracy: 0.9850 - loss: 7.0400 - val_accuracy: 0.0514 - val_loss: 9.8726
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9903 - loss: 5.7648 
Epoch 3: val_loss improved from 9.87264 to 8.95772, saving model to models/best_ResNet_model.keras
115/115 ━━━━━━━━━━━━━━━━━━━━ 1711s 15s/step - accuracy: 0.9903 - loss: 5.7621 - val_accuracy: 0.0724 - val_loss: 8.9577
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.9921 - loss: 4.5540 
Epoch 4

In [7]:
# Evaluate the model
model = tf.keras.models.load_model('models/best_ResNet_model.keras')
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f'Train accuracy: {train_acc:.4f}, Validation accuracy: {val_acc:.4f}, Test accuracy: {test_acc:.4f}')

2024-07-19 12:23:55.137293: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/jay/anaconda3/envs/tf_m1_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


115/115 ━━━━━━━━━━━━━━━━━━━━ 91s 757ms/step - accuracy: 0.9903 - loss: 0.1562
33/33 ━━━━━━━━━━━━━━━━━━━━ 29s 863ms/step - accuracy: 0.9653 - loss: 0.2472
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 821ms/step - accuracy: 0.9689 - loss: 0.2128
Train accuracy: 0.9910, Validation accuracy: 0.9681, Test accuracy: 0.9705


In [10]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()


AttributeError: 'Functional' object has no attribute 'history'

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

test_generator.reset()
Y_pred = model.predict(test_generator, len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))
cm = confusion_matrix(test_generator.classes, y_pred)
print(cm)

33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step
                   precision    recall  f1-score   support

     agricultural       0.05      0.06      0.06        50
         airplane       0.04      0.04      0.04        50
  baseballdiamond       0.08      0.08      0.08        50
            beach       0.08      0.08      0.08        50
        buildings       0.05      0.06      0.05        50
        chaparral       0.03      0.02      0.02        50
 denseresidential       0.07      0.06      0.06        50
           forest       0.11      0.10      0.10        50
          freeway       0.06      0.06      0.06        50
       golfcourse       0.08      0.08      0.08        50
           harbor       0.08      0.08      0.08        50
     intersection       0.04      0.04      0.04        50
mediumresidential       0.08      0.08      0.08        50
   mobilehomepark       0.04      0.04      0.04        50
         overpass       0.08      0.08      0.08        50
       parkinglo

In [ ]:
model.load_model('models/best_ResNet_model.keras')

train_loss, train_acc = model.evaluate(train_generator)
print(f"train_loss:{train_loss}:: train_accuracy: {train_acc}")

val_loss, val_acc = model.evaluate(val_generator)
print(f"val_loss:{val_loss}:: val_acc: {val_acc}")

test_loss, test_acc = model.evaluate(test_generator)
print(f"test_loss:{test_loss}:: test_acc: {test_acc}")
    
predictions = model.predict(test_generator)
predicted_label = np.argmax(predictions, axis=1)

class_names = ["Agricultural", "Airplane", "Baseballdiamond", "Beach", "Buildings", "Chaparral", "Denseresidential", 
               "Forest", "Freeway", "Golfcourse", "Harbor", "Intersection", "Mediumresidential", "Mobilehomepark", 
               "Overpass", "Parkinglot", "River", "Runway", "Sparseresidential", "Storagetanks", "Tenniscourt"
               ]

true_labels = []
for image, label in test_generator: #.as_numpy_iterator()
    true_labels += list(label.numpy())

# Plot confusion Matrix
ModelUtils.plot_cm(y_true=true_labels, y_pred=predicted_label, class_names=class_names, save_path=cfg.Path.figure_save_path)
# Plot Roc Auc Curve
ModelUtils.multiclass_roc_auc_score(y_true=true_labels, model_predicted_label=predicted_label, class_names=class_names, save_path=cfg.Path.figure_save_path)
